# Initial Set-up to perform Looker-sdk tasks

## <font color="rgb(250, 128, 114)"> Execute the cell to install the following packages if they were not installed already </font>
<p>
    <ul>
        <li> <span style="font-family:Comic Sans MS"> <font color="blue"> <i> ipyfilechooser </i> </font> </span> </li>
        <li> <span style="font-family:Comic Sans MS"> <font color="blue"> <i> looker-sdk </i> </font> </span> </li>
     </ul>
</p>

In [1]:
import ipywidgets as widgets

button = widgets.Button(description="Install Packages")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        !pip install ipyfilechooser
        !pip install looker-sdk

button.on_click(on_button_clicked)

Button(description='Install Packages', style=ButtonStyle())

Output()

<h2> <font color="rgb(250, 128, 114)"> Create two looker.ini files </font> </h2>
<h3> <font color="blue"> with SubsLookerPrd key and   </font> </h3>
<h3> <font color="blue"> with Admin key               </font> </h3>
<h3> <font color="blue"> Sample "looker.ini" details: </font> </h3>
<blockquote><p> </font> </p>

<blockquote>
<p> <b> <span style="font-family:Courier New">
[Looker]<br>
base_url=https://xpologisticsdev.cloud.looker.com:443<br>
client_id=<font color="blue">your Client ID</font><br>
client_secret=<font color="blue">your Client Secret ID</font><br>
verify_ssl=True<br>
</span> </b></p>
</blockquote>
</blockquote>

# Reassign the Subscriptions to "SubsLookerPrd"

## <font color="rgb(250, 128, 114)"> Execute the below Explore Query link and extract the information in xls: </font>

<blockquote> <br>
<nav>
<a href="https://xpologisticsprod.cloud.looker.com/explore/system__activity/scheduled_plan?toggle=fil&qid=UIc2lPPBUCRSjM71BevzIK"> Scheduled plan extraction from Looker </a>  <strong> => Make sure to adjust the filters appropriately </strong>
</nav>
<br>
<blockquote>
    This spreadsheet is then used to Reassign the Scheduled Plan
</blockquote>
</blockquote>

## <font color="rgb(250, 128, 114)"> Choose the Spreadsheet / File that contain the required informtion: </font>

In [10]:
import pandas as pd
import getpass
import json
import datetime
from ipyfilechooser import FileChooser

LookerSchedPlanFC = FileChooser('c://users//')
LookerSchedPlanFC.title = '<b> Choose the File that contains Looker Scheduled Plan details: </b>'
LookerSchedPlanFC.use_dir_icons = True
display(LookerSchedPlanFC)

LookerIniFC = FileChooser('c://users//')
LookerIniFC.title = '<b> Choose the Looker Init file (make sure to pass Admin key): </b>'
LookerIniFC.use_dir_icons = True
display(LookerIniFC)

OutputDirFC = FileChooser('c://users//')
OutputDirFC.title = '<b> Choose the Directory to store the Output files: </b>'
OutputDirFC.use_dir_icons = True
display(OutputDirFC)

FileChooser(path='C:\Users', filename='', title='<b> Choose the File that contains Looker Scheduled Plan detai…

FileChooser(path='C:\Users', filename='', title='<b> Choose the Looker Init file (make sure to pass Admin key)…

FileChooser(path='C:\Users', filename='', title='<b> Choose the Directory to store the Output files: </b>', sh…

## <font color="rgb(250, 128, 114)"> Execute the cell to process the file and Re-Assign the Subscriptions in Looker </font>

In [ ]:
import pandas as pd
import looker_sdk
import datetime
import ipywidgets as widgets

LookerSchedPlanFile   = LookerSchedPlanFC.selected.replace("\\", "\\\\")
LookerIniFile         = LookerIniFC.selected.replace("\\", "\\\\")
sdk                   = looker_sdk.init40(LookerIniFile)
layout                = widgets.Layout(width='400px', height='40px')
style                 = {'description_width': 'initial'}

LookerIniFile           = LookerIniFC.selected.replace("\\", "\\\\")
sdk                     = looker_sdk.init40(LookerIniFile)

LookerSchedPlanFile     = LookerSchedPlanFC.selected.replace("\\", "\\\\")
OutputDir               = OutputDirFC.selected.replace("\\", "\\\\")

# Load the Looker Schedule Plan details and Flatten the Address for the corresponding Subscription ID.
LookerSchedPlandf = pd.read_excel(LookerSchedPlanFile, sheet_name=0, header=0)
LookerSchedPlandf = LookerSchedPlandf.fillna(' ')


col_list          = []
for col in LookerSchedPlandf.columns:
    col_list.append(col)
    
def ReassignScheduledPlan(btn):
    PlanID = PlanIdSelection.value
    reassign_df         = pd.DataFrame()
    
    print('Process Started    : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    body = {}
    body['user_id'] = 357
    
    for idx, rows in LookerSchedPlandf.iterrows():
        #print(sdk.scheduled_plan(rows[PlanID]))
        exception = False
        status    = ""
        
        try:
            ReassignSchedPlanResp = sdk.update_scheduled_plan(rows[PlanID], body)
            status                = "Success"
        except Exception as e:
            exception             = True
            ReassignSchedPlanResp = e
            status                = "Failed"

        temp_df = pd.DataFrame({'PlanID':rows[PlanID], 'Status': status}, index=[0])
        reassign_df = pd.concat([reassign_df, temp_df])
    
    BackupFile   = 'Reassign_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx'
    reassign_df.to_excel('Reassign_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx', index=False, index_label=None)
    print("Check the SDK response in the Backupfile file: " + BackupFile)
    
    otputFile    = OutputDir + 'Create_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx'

    reassign_df.to_excel(otputFile, index=False, index_label=None)
    print("Check the SDK response in the Output file: " + otputFile)

    print('Process Completed  : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    

PlanIdSelection = widgets.Dropdown(
                                    options=col_list,
                                    description='Choose the column that contains Plan ID:',
                                    disabled=False,
                                    layout=layout,
                                    style=style,
                                  )
ProcessButton   = widgets.Button(description='Reassign')
ProcessButton.style.button_color = 'lightblue'


display(PlanIdSelection, ProcessButton)
ProcessButton.on_click(ReassignScheduledPlan)

# Re-Creating Subscriptions in SubsLookerPrd from Back-up

## <font color="rgb(250, 128, 114)"> Execute the below Explore Query link and extract the information in xls: </font>

<blockquote> <br>
<nav>
<a href="https://xpologisticsprod.cloud.looker.com/explore/system__activity/scheduled_plan?toggle=fil&qid=UIc2lPPBUCRSjM71BevzIK"> Scheduled plan extraction from Looker </a>  <strong> => Make sure to adjust the filters appropriately </strong>
</nav>
<br>
</blockquote>

## <font color="rgb(250, 128, 114)"> Choose the Spreadhseets / Files that contain the required information </font>

In [4]:
import pandas as pd
import getpass
import json
import datetime
from ipyfilechooser import FileChooser

LookerSchedPlanFC = FileChooser('c://users//')
LookerSchedPlanFC.title = '<b> Choose the File that contains Looker Scheduled Plan details: </b>'
LookerSchedPlanFC.use_dir_icons = True
display(LookerSchedPlanFC)

CreateSchedFC = FileChooser('c://users//')
CreateSchedFC.title = '<b> Choose the json file for creating Scheduled Plan: </b>'
CreateSchedFC.use_dir_icons = True
display(CreateSchedFC)

LookerIniFC = FileChooser('c://users//')
LookerIniFC.title = '<b> Choose the Looker Init file (make sure to pass SubsLookerPrd key): </b>'
LookerIniFC.use_dir_icons = True

display(LookerIniFC)

OutputDirFC = FileChooser('c://users//')
OutputDirFC.title = '<b> Choose the Directory to store the Output files: </b>'
OutputDirFC.use_dir_icons = True
display(OutputDirFC)

FileChooser(path='C:\Users', filename='', title='<b> Choose the File that contains Looker Scheduled Plan detai…

FileChooser(path='C:\Users', filename='', title='<b> Choose the json file for creating Scheduled Plan: </b>', …

FileChooser(path='C:\Users', filename='', title='<b> Choose the Looker Init file (make sure to pass SubsLooker…

FileChooser(path='C:\Users', filename='', title='<b> Choose the Directory to store the Output files: </b>', sh…

## <font color="rgb(250, 128, 114)"> Execute the cell to process the file and set-up the Subscriptions in Looker </font>

In [8]:
import looker_sdk

print('Process Started    : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

LookerIniFile     = LookerIniFC.selected.replace("\\", "\\\\")
sdk               = looker_sdk.init40(LookerIniFile)

def to_list(col):
    return list(col)

LookerSchedPlanFile     = LookerSchedPlanFC.selected.replace("\\", "\\\\")
CreateSchedFile         = CreateSchedFC.selected.replace("\\", "\\\\")
OutputDir               = OutputDirFC.selected.replace("\\", "\\\\")

# Load the Looker Schedule Plan details and Flatten the Address for the corresponding Subscription ID.
LookerSchedPlandf = pd.read_excel(LookerSchedPlanFile, sheet_name=0, header=0)
LookerSchedPlandf = LookerSchedPlandf.fillna(' ')

Columns = LookerSchedPlandf.columns.tolist()
Columns.remove('Scheduled Plan Destination Address')

# Remove the schedules that doesn't have CronTab or DataGroup
#LookerSchedPlandf = LookerSchedPlandf.loc[(LookerSchedPlandf['Scheduled Plan Cron Schedule'] != " ") | (LookerSchedPlandf['Scheduled Plan Datagroup'] != " ")]

# Grouping the Dataframes and putting the address in the list
LookerSchedPlandf = LookerSchedPlandf.groupby(Columns).agg({'Scheduled Plan Destination Address': to_list}).reset_index()

def BuildAddress(var):
    
    EmailAddress = ""
    for EmailID in var:
        EmailAddress = EmailAddress + ", " + EmailID.strip()
    return EmailAddress.lstrip(' ').lstrip(',').lstrip(' ')

final = pd.DataFrame()

ParamCheck = lambda x: False if x == "No" else True

for idx, rows in LookerSchedPlandf.iterrows():
    
    with open(CreateSchedFile, 'r') as json_file:
        CreateSchedPlan                        = json.load(json_file)
        CreateSchedPlan['name']                = rows['Scheduled Plan Name']
        CreateSchedPlan['user_id']             = 1201  # Comment this line for <Testing>
        CreateSchedPlan['run_as_recipient']    = ParamCheck(rows['Scheduled Plan Run As Recipient (Yes / No)']) 
        CreateSchedPlan['enabled']             = ParamCheck(rows['Scheduled Plan Enabled (Yes / No)']) 
        CreateSchedPlan['lookml_dashboard_id'] = None
        CreateSchedPlan['filters_string']      = rows['Scheduled Plan Filters String']
        CreateSchedPlan['require_results']     = ParamCheck(rows['Scheduled Plan Require Results (Yes / No)']) 
        CreateSchedPlan['require_no_results']  = ParamCheck(rows['Scheduled Plan Require No Results (Yes / No)']) 
        CreateSchedPlan['require_change']      = ParamCheck(rows['Scheduled Plan Require Change (Yes / No)']) 
        CreateSchedPlan['send_all_results']    = ParamCheck(rows['Scheduled Plan Send All Results (Yes / No)']) 
        CreateSchedPlan['timezone']            = rows['Scheduled Plan Timezone']
        CreateSchedPlan['query_id']            = None
        CreateSchedPlan['run_once']            = ParamCheck(rows['Scheduled Plan Run Once (Yes / No)'])
        CreateSchedPlan['include_links']       = ParamCheck(rows['Scheduled Plan Include Links (Yes / No)']) 
        CreateSchedPlan['pdf_paper_size']      = None
        CreateSchedPlan['pdf_landscape']       = False
        CreateSchedPlan['embed']               = False
        CreateSchedPlan['color_theme']         = "~default"
        CreateSchedPlan['long_tables']         = False
        CreateSchedPlan['inline_table_width']  = None


        CreateSchedPlan['scheduled_plan_destination'][0]['id']                = 0
        CreateSchedPlan['scheduled_plan_destination'][0]['scheduled_plan_id'] = 0
        CreateSchedPlan['scheduled_plan_destination'][0]['format']            = rows['Scheduled Plan Destination Format']
        CreateSchedPlan['scheduled_plan_destination'][0]['apply_formatting']  = ParamCheck(rows['Scheduled Plan Destination Apply Formatting (Yes / No)']) 
        CreateSchedPlan['scheduled_plan_destination'][0]['apply_vis']         = ParamCheck(rows['Scheduled Plan Destination Apply Vis (Yes / No)']) 
        CreateSchedPlan['scheduled_plan_destination'][0]['address']           = BuildAddress(rows['Scheduled Plan Destination Address'])
        CreateSchedPlan['scheduled_plan_destination'][0]['type']              = rows['Scheduled Plan Destination Type']
        CreateSchedPlan['scheduled_plan_destination'][0]['parameters']        = None
        CreateSchedPlan['scheduled_plan_destination'][0]['secret_parameters'] = None
        CreateSchedPlan['scheduled_plan_destination'][0]['message']           = rows['Scheduled Plan Destination Message']

        if rows['Look ID'] != " ":
            CreateSchedPlan['look_id'] = rows['Look ID']
        else:
            CreateSchedPlan['look_id'] = None

        if rows['Dashboard ID (User-defined only)'] != " ":
            CreateSchedPlan['dashboard_id'] = rows['Dashboard ID (User-defined only)']
        else:
            CreateSchedPlan['dashboard_id'] = None

        if rows['Scheduled Plan Cron Schedule'] != " ":
            CreateSchedPlan['crontab'] = rows['Scheduled Plan Cron Schedule']  #"30 12 29 2 *" # use this CronTab for <Testing>
        else:
            CreateSchedPlan['crontab'] = None

        if rows['Scheduled Plan Datagroup'] != " ":
            CreateSchedPlan['datagroup'] = rows['Scheduled Plan Datagroup']
        else:
            CreateSchedPlan['datagroup'] = None

        CreateSchedPlanjson = json.dumps(CreateSchedPlan)
        #print(CreateSchedPlanjson) # Uncomment for <Testing>

#Comment all the lines below for <Testing> until the end tag "</End>"
#<Start>
    exception = False
    status    = ""
    try:
        CreateSchedPlanResp = sdk.create_scheduled_plan(CreateSchedPlanjson)
        status = "Success"
    except Exception as e:
        exception = True
        CreateSchedPlanResp = e
        status = "Failed"
    if exception == False:
        TmpSchedPlanID = str(CreateSchedPlanResp['scheduled_plan_destination'][0])
        TmpSchedPlanID = TmpSchedPlanID[TmpSchedPlanID.find('scheduled_plan_id') + 18: 60]
        TmpSchedPlanID = TmpSchedPlanID[0: TmpSchedPlanID.find(',')]
    else:
        TmpSchedPlanID = None

    df = pd.DataFrame({'SchedName': rows['Scheduled Plan Name'], \
                       'SchedulePlanID': TmpSchedPlanID, 'TimeZone': rows['Scheduled Plan Timezone'], \
                       'Status': status, \
                       'LookID': rows['Look ID'], \
                       'DashboardID': rows['Dashboard ID (User-defined only)'], \
                       'CronTab': rows['Scheduled Plan Cron Schedule'], \
                       'DataGroup': rows['Scheduled Plan Datagroup'], \
                       'Format': rows['Scheduled Plan Destination Format'], \
                       'Type': rows['Scheduled Plan Destination Type'], \
                       'EmailBody': rows['Scheduled Plan Destination Message'], \
                       'EmailAddress': BuildAddress(rows['Scheduled Plan Destination Address']), \
                       'FilterString': rows['Scheduled Plan Filters String'], \
                       'CreateSchedPlanReq': CreateSchedPlanjson, 'CreateSchedPlanResp': str(CreateSchedPlanResp) \
                      }, index=[0])


    final = pd.concat([final, df])

BackupFile   = 'Create_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx'
final.to_excel(BackupFile, index=False, index_label=None)
print("Check the SDK response in the Backupfile file: " + BackupFile)
    
otputFile    = OutputDir + 'Create_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx'

final.to_excel(otputFile, index=False, index_label=None)
print("Check the SDK response in the Output file: " + otputFile)

#</End>

print('Process Completed  : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    


Process Started    : 2022-03-01 21:15:36
Check the SDK response in the Backupfile file: Create_Schedule_Plan_2022-03-01 21_15_44.xlsx
Check the SDK response in the Output file: C:\\Users\\mratna\\Documents\\XPO\\CX\\Subs Auto\\Create_Schedule_Plan_2022-03-01 21_15_44.xlsx
Process Completed  : 2022-03-01 21:15:44


## <font color="rgb(250, 128, 114)"> Choose the files to delete the Scheduled plan: </font>

# Instructions for Deleting the Scheduled Plans

In [5]:
import getpass
from ipyfilechooser import FileChooser

DeleteSchedFC = FileChooser('c://users//')
DeleteSchedFC.title = '<b> Choose the file that contains Scheduled Plan ID: </b>'
DeleteSchedFC.use_dir_icons = True
display(DeleteSchedFC)

LookerIniFC = FileChooser('c://users//')
LookerIniFC.title = '<b> Choose the Looker Init file (make sure to pass admin key): </b>'
LookerIniFC.use_dir_icons = True
display(LookerIniFC)

FileChooser(path='C:\Users', filename='', title='<b> Choose the file that contains Scheduled Plan ID: </b>', s…

FileChooser(path='C:\Users', filename='', title='<b> Choose the Looker Init file (make sure to pass admin key)…

## <font color="rgb(250, 128, 114)"> Execute the cell to Delete the Looker Subscriptions: </font>

In [6]:
import pandas as pd
import looker_sdk
import datetime
import ipywidgets as widgets

DeleteSchedFile   = DeleteSchedFC.selected.replace("\\", "\\\\")
LookerIniFile     = LookerIniFC.selected.replace("\\", "\\\\")
sdk               = looker_sdk.init40(LookerIniFile)
layout            = widgets.Layout(width='400px', height='40px')
style             = {'description_width': 'initial'}
delete_df         = pd.DataFrame()
col_list          = []

DeleteScheddf = pd.read_excel(DeleteSchedFile, sheet_name=0, header=0)

for col in DeleteScheddf.columns:
    col_list.append(col)
    
def GetScheduledPlan(btn):
    PlanID = PlanIdSelection.value
    delete_df         = pd.DataFrame()
    
    print('Process Started    : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    for idx, rows in DeleteScheddf.iterrows():
        #print(sdk.scheduled_plan(rows[PlanID]))
        try:
            DeleteSchedResp = sdk.delete_scheduled_plan(rows[PlanID])
            DeleteSchedResp = 'Subscription Deleted from Looker'
        except:
            DeleteSchedResp = 'Couldnt get deleted due to an error'

        temp_df = pd.DataFrame({'PlanID':rows[PlanID], 'SDKResp': DeleteSchedResp}, index=[0])
        delete_df = pd.concat([delete_df, temp_df])
    
    delete_df.to_excel('Delete_Schedule_Plan_' + datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S") + '.xlsx', index=False, index_label=None)

    print('Process Completed  : ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    

PlanIdSelection = widgets.Dropdown(
                                    options=col_list,
                                    description='Choose the column that contains Plan ID:',
                                    disabled=False,
                                    layout=layout,
                                    style=style,
                                  )
ProcessButton   = widgets.Button(description='Del Scheduled Plan')
ProcessButton.style.button_color = 'lightblue'


display(PlanIdSelection, ProcessButton)
ProcessButton.on_click(GetScheduledPlan)

Dropdown(description='Choose the column that contains Plan ID:', layout=Layout(height='40px', width='400px'), …

Button(description='Del Scheduled Plan', style=ButtonStyle(button_color='lightblue'))

Process Started    : 2022-03-01 17:49:51
Process Completed  : 2022-03-01 17:49:54


In [ ]:
LookerSchedPlandf
CreateSchedPlan['crontab']